In [1]:
import pandas as pd

train = pd.read_csv("train_users.csv")
test = pd.read_csv("test_users.csv")
sessions = pd.read_csv("sessions.csv")

In [2]:
for data in (train, test):
    data['year_created']  = data['date_account_created'].apply(lambda x: 'nan' if str(x) == 'nan' else int(str(x)[:4]) )
    data['month_created'] = data['date_account_created'].apply(lambda x: 'nan' if str(x) == 'nan' else int(str(x)[5:7]))
    data['week_created']  = data['date_account_created'].apply(lambda x: 'nan' if str(x) == 'nan' else int(str(x)[8:10]))
    
    data['year_first']  = data['date_first_booking'].apply(lambda x: 'nan' if str(x) == 'nan' else int(str(x)[:4]))
    data['month_first'] = data['date_first_booking'].apply(lambda x: 'nan' if str(x) == 'nan' else int(str(x)[5:7]))
    data['week_first']  = data['date_first_booking'].apply(lambda x: 'nan' if str(x) == 'nan' else int(str(x)[8:10]))

In [3]:
for data in (train, test):
    data.drop(['date_account_created'], axis=1,inplace=True)
    data.drop(['date_first_booking'], axis=1,inplace=True)

In [4]:
train = pd.merge(train, sessions, how="left", left_on=["id"], right_on=["user_id"])
test = pd.merge(test, sessions, how="left", left_on=["id"], right_on=["user_id"])

In [5]:
countries = pd.read_csv("countries.csv")

In [6]:
train = pd.merge(train, countries, how="left", left_on=["country_destination"], right_on=["country_destination"])

In [18]:
train.columns.summary

<bound method Index.summary of Index([u'id', u'timestamp_first_active', u'gender', u'age', u'signup_method', u'signup_flow', u'language', u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked', u'signup_app', u'first_device_type', u'first_browser', u'country_destination', u'year_created', u'month_created', u'week_created', u'year_first', u'month_first', u'week_first', u'user_id', u'action', u'action_type', u'action_detail', u'device_type', u'secs_elapsed', u'lat_destination', u'lng_destination', u'distance_km', u'destination_km2', u'destination_language ', u'language_levenshtein_distance'], dtype='object')>

In [20]:
x_train = train.drop("country_destination", axis=1)
y_train = train["country_destination"]
x_test = test.copy()

In [21]:
import numpy as np

from sklearn import preprocessing

for f in x_train.columns:
    if x_train[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        if f not in x_test.columns:
            lbl.fit(np.unique(list(x_train[f].values)))
            x_train[f] = lbl.transform(list(x_train[f].values))
        else:
            lbl.fit(np.unique(list(x_train[f].values) + list(x_test[f].values)))
            x_train[f] = lbl.transform(list(x_train[f].values))
            x_test[f] = lbl.transform(list(x_test[f].values))

In [22]:
x_test.columns

Index([u'id', u'timestamp_first_active', u'gender', u'age', u'signup_method', u'signup_flow', u'language', u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked', u'signup_app', u'first_device_type', u'first_browser', u'year_created', u'month_created', u'week_created', u'year_first', u'month_first', u'week_first', u'user_id', u'action', u'action_type', u'action_detail', u'device_type', u'secs_elapsed'], dtype='object')

In [23]:
x_train.columns

Index([u'id', u'timestamp_first_active', u'gender', u'age', u'signup_method', u'signup_flow', u'language', u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked', u'signup_app', u'first_device_type', u'first_browser', u'year_created', u'month_created', u'week_created', u'year_first', u'month_first', u'week_first', u'user_id', u'action', u'action_type', u'action_detail', u'device_type', u'secs_elapsed', u'lat_destination', u'lng_destination', u'distance_km', u'destination_km2', u'destination_language ', u'language_levenshtein_distance'], dtype='object')

In [24]:
for col in countries.columns:
    if col == 'country_destination':
        continue
    del(x_train[col])

In [25]:
x_test.columns

Index([u'id', u'timestamp_first_active', u'gender', u'age', u'signup_method', u'signup_flow', u'language', u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked', u'signup_app', u'first_device_type', u'first_browser', u'year_created', u'month_created', u'week_created', u'year_first', u'month_first', u'week_first', u'user_id', u'action', u'action_type', u'action_detail', u'device_type', u'secs_elapsed'], dtype='object')

In [27]:
x_train.columns

Index([u'id', u'timestamp_first_active', u'gender', u'age', u'signup_method', u'signup_flow', u'language', u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked', u'signup_app', u'first_device_type', u'first_browser', u'year_created', u'month_created', u'week_created', u'year_first', u'month_first', u'week_first', u'user_id', u'action', u'action_type', u'action_detail', u'device_type', u'secs_elapsed'], dtype='object')

In [28]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
x_train_nonan = imp.fit_transform(x_train)


x_test_nonan = imp.fit_transform(x_test)

In [29]:
x_train

,id,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,...,week_created,year_first,month_first,week_first,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,129855,20090319043255,0,NaN,1,0,6,2,4,7,...,28,6,12,31,87710,189,7,81,14,NaN
1,61843,20090523174809,2,38,1,0,6,7,8,7,...,25,6,12,31,87710,189,7,81,14,NaN
2,188750,20100105051812,1,47,0,0,6,3,2,7,...,5,6,12,31,87710,189,7,81,14,NaN
3,106886,20100105083259,1,37,0,0,6,3,2,0,...,5,6,12,31,87710,189,7,81,14,NaN
4,152148,20100107055820,1,36,0,0,6,3,2,7,...,7,6,12,31,87710,189,7,81,14,NaN
5,18310,20100107215125,0,NaN,0,0,6,3,2,3,...,7,6,12,31,87710,189,7,81,14,NaN
6,99093,20100108015641,0,NaN,0,0,6,7,6,3,...,8,6,12,31,87710,189,7,81,14,NaN
7,149422,20100111224015,0,NaN,0,0,6,3,2,7,...,11,6,12,31,87710,189,7,81,14,NaN
8,57659,20100111230808,1,40,0,0,6,7,8,7,...,11,6,12,31,87710,189,7,81,14,NaN
9,88325,20100112155420,0,NaN,0,0,6,3,12,7,...,12,6,12,31,87710,189,7,81,14,NaN


In [30]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()


model.fit(x_train_nonan, y_train)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=None, splitter='best')

In [31]:
y_test = model.predict(x_test_nonan)

In [32]:
submission = pd.DataFrame()
submission["id"] = test["id"]
submission["country"] = y_test

submission.to_csv('airbnb_1.csv', index=False)